Data Preprocessing

In [ ]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch as t

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [ ]:
!pip install -U torchtext==0.8.0

In [ ]:
!pip install transformers

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
np.random.seed(500)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_train.csv")
df2 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_test.csv")
df3 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/both_val.csv")
df4 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/schizoaffective.csv")
df5 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/AlAnon.csv")
df6 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/BorderlinePDisorder_10.csv")
df7 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/alcoholicsanonymous.csv")
df8 = pd.read_csv("/content/drive/MyDrive/CIS 519 Final Project/OCD.csv")

In [ ]:
df4.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df5.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df6.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df7.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df8.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)

In [ ]:
df4 = df4[df4["body"] != "[deleted]"]
df5 = df5[df5["body"] != "[deleted]"]
df6 = df6[df6["body"] != "[deleted]"]
df7 = df7[df7["body"] != "[deleted]"]
df8 = df8[df8["body"] != "[deleted]"]

In [ ]:
df4["class_id"] = [6]*len(df4)
df5["class_id"] = [7]*len(df5)
df6["class_id"] = [8]*len(df6)
df7["class_id"] = [7]*len(df7)
df8["class_id"] = [9]*len(df8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df4.dropna(inplace=True)
df5.dropna(inplace=True)
df6.dropna(inplace=True)
df7.dropna(inplace=True)
df8.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df4.rename(columns={"body":"post","id":"ID"}, inplace=True)
df5.rename(columns={"body":"post","id":"ID"}, inplace=True)
df6.rename(columns={"body":"post","id":"ID"}, inplace=True)
df7.rename(columns={"body":"post","id":"ID"}, inplace=True)
df8.rename(columns={"body":"post","id":"ID"}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8]
corpus = pd.concat(frames, ignore_index=True)

In [ ]:
corpus

,ID,title,post,class_name,class_id
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5
...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9


In [ ]:
#df1.value_counts()

In [ ]:
corpus["text"] = corpus.title + " " + corpus.post

In [ ]:
corpus

,ID,title,post,class_name,class_id,text
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5,a question about the third conditional. i was ...
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0,the epitome of my life i've recently requested...
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5,what are your favourites offbeat destinations ...
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5,synesthesia survey (what colour is each month ...
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5,"science ama series: i’m phil baran, and i’m he..."
...,...,...,...,...,...,...
23572,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9,Can you have ocd without having depression??? ...
23573,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9,"Hi! I'm an OCD therapist, and I'm new to reddi..."
23574,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9,Vent: my mom who I still live with tested posi...
23575,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9,Do “intrusive memories” count as intrusive tho...


In [ ]:
corpus1 = corpus[["text","class_id"]]

In [ ]:
corpus1

,text,class_id
0,a question about the third conditional. i was ...,5
1,the epitome of my life i've recently requested...,0
2,what are your favourites offbeat destinations ...,5
3,synesthesia survey (what colour is each month ...,5
4,"science ama series: i’m phil baran, and i’m he...",5
...,...,...
23572,Can you have ocd without having depression??? ...,9
23573,"Hi! I'm an OCD therapist, and I'm new to reddi...",9
23574,Vent: my mom who I still live with tested posi...,9
23575,Do “intrusive memories” count as intrusive tho...,9


In [ ]:
corpus1["class_id"].value_counts()

7    4985
0    2961
3    2946
1    2918
2    2903
4    2497
5    2478
9     787
8     639
6     463
Name: class_id, dtype: int64

XLNet

In [ ]:
pip install transformers

In [ ]:
pip install simpletransformers

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


# Create a ClassificationModel
model = MultiLabelClassificationModel('xlnet', 'xlnet-base-cased', num_labels=10)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream tas

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
# Create a TransformerModel with modified attributes
from simpletransformers.model import TransformerModel
model = TransformerModel('xlnet', 'xlnet-base-cased', num_labels=10, args={'learning_rate':1e-5, 'num_train_epochs': 2, 'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
corpus1

,text,class_id
0,a question about the third conditional. i was ...,5
1,the epitome of my life i've recently requested...,0
2,what are your favourites offbeat destinations ...,5
3,synesthesia survey (what colour is each month ...,5
4,"science ama series: i’m phil baran, and i’m he...",5
...,...,...
23572,Can you have ocd without having depression??? ...,9
23573,"Hi! I'm an OCD therapist, and I'm new to reddi...",9
23574,Vent: my mom who I still live with tested posi...,9
23575,Do “intrusive memories” count as intrusive tho...,9


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(corpus1, test_size=0.2)


In [ ]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/18861 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2358 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 2:   0%|          | 0/2358 [00:00<?, ?it/s]

(4716, 0.5674854529242681)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4716 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/590 [00:00<?, ?it/s]

In [ ]:
print(result)
print(model_outputs)
print(wrong_predictions)

{'mcc': 0.847306286778114, 'f1': 0.8674724342663274, 'acc': 0.8674724342663274, 'eval_loss': 0.4694232623455888}
[[-1.87109375 -0.99755859 -0.21069336 ... -1.29296875  7.2109375
  -0.41259766]
 [-2.21679688 -2.22070312 -0.09747314 ... -0.49316406  1.69628906
   1.32714844]
 [-0.23242188 -0.60449219 -0.40454102 ... -0.21228027 -2.1015625
  -1.93261719]
 ...
 [-3.3203125   2.7421875  -2.57421875 ...  2.68945312  1.33789062
   2.73632812]
 [-1.53515625 -1.45214844 -0.84765625 ...  9.2890625  -0.48168945
  -0.78808594]
 [-1.22949219 -1.31445312 -0.70849609 ...  9.4765625  -0.15844727
  -0.13793945]]
[]
